In [1]:
import pandas as pd
import os
import re
import tqdm
from typing import List, Dict
import json
import shutil

In [2]:
import stanza
import pymorphy3
import pymorphy2

nlp = stanza.Pipeline('uk', processors='tokenize,mwt,pos,lemma,depparse')
morph = pymorphy3.MorphAnalyzer()


# # stanza.download('uk')

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2025-04-04 23:24:07 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-04 23:24:07 INFO: Downloaded file to /Users/linndfors/stanza_resources/resources.json
2025-04-04 23:24:09 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| pos       | iu_charlm   |
| lemma     | iu_nocharlm |
| depparse  | iu_charlm   |

2025-04-04 23:24:09 INFO: Using device: cpu
2025-04-04 23:24:09 INFO: Loading: tokenize
2025-04-04 23:24:10 INFO: Loading: mwt
2025-04-04 23:24:10 INFO: Loading: pos
2025-04-04 23:24:15 INFO: Loading: lemma
2025-04-04 23:24:16 INFO: Loading: depparse
2025-04-04 23:24:17 INFO: Done loading processors!


In [3]:
parallel_dataset_ng_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")
parallel_dataset_bruk_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv")

In [4]:
parallel_dataset_bruk_dataset.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
0,Його редактором був поет-символіст Яків Савчен...,46,Його редакторкою була поетеса-символістка Анже...,46,e5e76a8efa0f.txt,"{'T30': ('поет-символіст', 'JOB')}","{'T30': ('поетеса-символістка', 'JOB')}"
1,Міжпредметні паралелі . Маніфест футуристів ск...,62,Міжпредметні паралелі . Маніфест футуристок ск...,62,e5e76a8efa0f.txt,"{'T49': ('поет', 'JOB')}","{'T49': ('поетеса', 'JOB')}"
2,Помітною була організація « Гарт » ( 1923 — 19...,96,Помітною була організація « Гарт » ( 1923 — 19...,96,e5e76a8efa0f.txt,"{'T104': ('поет', 'JOB')}","{'T104': ('поетка', 'JOB')}"
3,Її очолив байкар і прозаїк Сергій Пилипенко .,127,Її очолила байкарка і прозаїкиня Марія Пилипен...,127,e5e76a8efa0f.txt,"{'T143': ('байкар', 'JOB'), 'T144': ('прозаїк'...","{'T143': ('байкарка', 'JOB'), 'T144': ('прозаї..."
4,У Галицько-Волинському літописі згадується спі...,171,У Галицько-Волинському літописі згадується спі...,171,e5e76a8efa0f.txt,"{'T218': ('співець', 'JOB')}","{'T218': ('співчиня', 'JOB')}"


# Split: Female, Male, Common genders

In [5]:
with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/common_gender_words_list.txt') as file:
    common_gender_words_list = [line.strip() for line in file if line.strip()]

with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/male_words_list.txt') as file:
    male_list = [line.strip() for line in file if line.strip()]

with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/female_words_list.txt') as file:
    female_list = [line.strip() for line in file if line.strip()]

gender_dict_df = pd.read_csv("/Users/linndfors/study/diploma/uk-gender-word-mapper/gender_pairs_dictionary.csv")

gender_dict = {'male': [], 'female': []}

for _, row in gender_dict_df.iterrows():
    gender_dict['male'].append(row['male'])
    female_values = [f.strip() for f in row['female'].split(',')]
    gender_dict['female'].extend(female_values)

gender_dict['female'] = list(set(gender_dict['female']))

exceptions_common = ["судді",  "глава", "голова", "керівництво", "в. о.", "головою"]
exceptions_male = ['ієромонах', 'прокурор', 'віце-премʼєр', 'премʼєр', 'начальник', 'міністр', 'директор', 'підрядник', 'генпідрядник', 'керівник', 'головнокомандувач']
exceptions_female = ['премʼєрка', 'докторка', 'докторантка', 'директорка', 'міністерка', 'керівниця', 'начальниця', 'начальницю', 'генпідрядниця', 'прокурорка', 'ректорка']

In [6]:
def parse_output(ent):
    noun_dict = {
        "хліборобок": "хліборобка",
    "культурологині": "культурологиня",
    "директоркою": "директорка",
    "слідчій": "слідча",
    "прокурорка": "прокурорка",
    "прокурорки": "прокурорка",
    "Прокурорки": "прокурорка",
    "прокуроркою": "прокурорка",
    "прокурорці": "прокурорка",
    "адміністраторок": "адміністраторка",
    "слідчу": "слідча",
    "слідча": "слідча",
    "начальницею": "начальниця",
    "податківка": "податківка",
    "монопольниці": "монопольниця",
    "начальницю": "начальниця",
    "психіатрині": "психіатриня",
    "наркологині": "наркологиня",
    "міністерки": "міністерка",
    "рітейлерками": "рітейлерка",
    "начальниці": "начальниця",
    "екологині": "екологиня",
    "податківчині": "податківчиня",
    "журналісткам": "журналістка",
    "підрядницею": "підрядниця",
    "директорка": "директорка",
    "інженерка": "інженерка",
    "службовиці": "службовиця",
    "інженерці": "інженерка",
    "податківниць": "податківниця",
    "депутатку": "депутатка",
    "керівниця": "керівниця",
    "керівницею": "керівниця",
    "юристок": "юристка",
    "інспекторки": "інспекторка",
    "інженерок": "інженерка",
    "ревізорок": "ревізорка",
    "нардепок": "нардепка",
    "бізнесменок": "бізнесменка",
    "генпідрядниці": "генпідрядниця",
    "прокурор": "прокурор",
    "віце-премʼєра": "віце-прем'єр",
    "премʼєр": "прем'єр",
    "начальника": "начальник",
    "начальником": "начальник",
    "міністра": "міністр",
    "директором": "директор",
    "підрядником": "підрядник",
    "аграрія": "аграрій",
    "депутата": "депутат",
    "генпідрядника": "генпідрядник",
    "генпідрядником": "генпідрядник",
    "начальник": "начальник",
    "керівником": "керівник",
    "керівник": "керівник",
    "судді": "суддя",
    "ченці": "чернець",
    "ченців": "чернець",
    "глави": "глава",
    "голови": "голова",
    "керівництво": "керівництво",
    "головою": "голова",
    "князів": "князь",
    "водія": "водій",
    "хіміка": "хімік",
    "хіміки": "хімік",
    "ведучій": "ведуча",
    "механіка": "механік",
    "інокам": "інок",
    "головнокомандувача": "головнокомандувач",
    "наркологині": "наркологиня",
    "членкині": "членкиня",
    "інженерці": "інженерка",
    "мисливствознавиці": "мисливствознавиця",
    "логопедині": "логопединя",
    "математикині": "математикиня",
    "психологині": "психологиня", 
    "філологині": "філологиня",
    "голопедині": "голопединя",
    "урядовицs": "урядовиця",
    "філософині": "філософиня",
    "педагогині": "педагогиня",
    "мера": "мер"
    }
    if ent in noun_dict.keys():
        return noun_dict[ent]
    return

In [7]:
def extract_main_word(text):
    if "-" in text:
        parts = text.split("-")
        main_word = parts[-1]
        doc = nlp(main_word)
        for sentence in doc.sentences:
            for word in sentence.words:
                return word.lemma

    doc = nlp(text)

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.head == 0:
                main_word = word
                if main_word.text in {"рок", "анти", "псевдо", "віце", "топ"}:
                    continue
                return main_word.lemma
    for sentence in doc.sentences:
        for word in sentence.words:
            return word.lemma
        
def extract_gender(entity):
    entity = entity.lower()
    job = nlp(entity)
    join_sign = " " if " " in entity else ("" if "-" in entity else "")
    job_ent_list = [word.lemma for sent in job.sentences for word in sent.words]
    job_lemma = join_sign.join(job_ent_list)

    if len(job_ent_list) > 1:
        job_lemma = extract_main_word(job_lemma)

    words = entity.split()
    lemmatized_words = []
    for word in words:
        parsed_word = morph.parse(word)[0]
        if 'plur' in parsed_word.tag:
            singular_form = parsed_word.inflect({'sing'}).word if parsed_word.inflect({'sing'}) else parsed_word.normal_form
            lemmatized_words.append(singular_form)
        else:
            lemmatized_words.append(parsed_word.normal_form)
            
    job_lemma_pymorphy = join_sign.join(lemmatized_words)

    if len(job_ent_list) > 1:
        job_lemma_pymorphy = extract_main_word(job_lemma_pymorphy)

    ent_forms = [entity, job_lemma, job_lemma_pymorphy]

    custom_dict_ent_value = parse_output(entity)
    if custom_dict_ent_value:
        ent_forms.append(custom_dict_ent_value)

    for word in words:
        for common_word in exceptions_common:
            if common_word == word or common_word == parse_output(word):
                return "common", job_lemma
            
        for female_word in exceptions_female:
            if female_word == word or female_word == parse_output(word):
                return "female", job_lemma
            
        for male_word in exceptions_male:
            if male_word == word or male_word == parse_output(word):
                return "male", job_lemma
        
    for x in ent_forms:
        if (x in gender_dict['female']) or (x in female_list) or ("знавиця" in x):
            return "female", job_lemma
        elif (x in gender_dict['male']) or (x in male_list) or ("знавець" in x) or (x=="мера"):
            return "male", job_lemma
        elif x in common_gender_words_list:
            return "common", job_lemma
        
    print("unkown for:", entity, " - ", job_lemma, " - ", job_lemma_pymorphy)
    return "unknown_gender", job_lemma

In [8]:
import ast

def return_gendered_dict(parallel_dataset, annotation_col_name, swapped=0):
    file_gender_dict = {"male": {}, "female": {}, "common": {}, "unknown_gender": {}}

    total_job_counter = 0
    job_list = []

    for x, row in tqdm.tqdm(parallel_dataset.iterrows()):
        filename = row['original_file_name']
        if swapped:
            filename = filename.replace(".txt", "_1.txt")
        orig_annotation = row[annotation_col_name]

        ann_str = orig_annotation.replace("'", '"')
        ann_str = ann_str.replace("–", '-')
        json_ann = ast.literal_eval(ann_str)
        
        try:
            for ent, feat in json_ann.items():
                if feat[1] == 'JOB':
                    total_job_counter += 1
                    job_list.append(feat[0])
                    gender_value, lemma_word = extract_gender(feat[0])
                    
                    if filename not in file_gender_dict[gender_value]:
                        file_gender_dict[gender_value][filename] = [(ent, lemma_word)]
                    else:
                        file_gender_dict[gender_value][filename].append((ent, lemma_word))
                    
        except Exception as e:
            print(f"Issue with row: {row} - Error: {e}")
    return file_gender_dict, total_job_counter, job_list

In [9]:
from collections import Counter

def return_gender_stat(total_job_counter, file_gender_dict):
    print("total size:", total_job_counter)

    for gender_class, val in file_gender_dict.items():
        print("================")
        
        print("gender:", gender_class)

        gender_entities = []

        print("number of files for the gender:", len(file_gender_dict[gender_class]))
        number_of_ents = 0
        for files, ents in file_gender_dict[gender_class].items():
            number_of_ents += len(ents)
            gender_entities += [pair[1] for pair in ents]
            
        print("number of entities for the gender:", number_of_ents)

        counter = Counter(gender_entities)

        print("The most popular entity:", counter.most_common(1)[0][0])

        print("================")

In [10]:
def split_files(file_gender_dict, source_dir):
    female_files = []
    male_files = []
    common_files = []

    for gen, file_with_ent in file_gender_dict.items():
        for filename, ent_with_ind in file_with_ent.items():
            full_path = os.path.join(source_dir, filename)
            if gen == "male":
                male_files.append(full_path)
            elif gen == "female":
                female_files.append(full_path)
            elif gen == "common":
                common_files.append(full_path)

    return female_files, male_files, common_files

## Original entitites

### Ng

In [12]:
orig_ng_file_gender_dict, orig_ng_total_job_counter, orig_ng_job_list = return_gendered_dict(parallel_dataset_ng_dataset, "orig_ann")

178it [01:05,  2.91it/s]

unkown for: податківці  -  податок  -  податківці


234it [01:32,  3.64it/s]

unkown for: _керівником  -  ткерівник  -  _керівник


311it [01:57,  1.78it/s]

...

973it [05:36,  4.49it/s]

unkown for: в. о. управління  -  в  -  в.


990it [05:40,  4.80it/s]

unkown for: віце-премʼєра  -  прем’єрний  -  прем’єрый


1021it [05:48,  2.93it/s]


In [13]:
return_gender_stat(orig_ng_total_job_counter, orig_ng_file_gender_dict)

total size: 1338
gender: male
number of files for the gender: 269
number of entities for the gender: 1134
The most popular entity: директор
gender: female
number of files for the gender: 16
number of entities for the gender: 25
The most popular entity: підприємиця
gender: common
number of files for the gender: 85
number of entities for the gender: 161
The most popular entity: голова
gender: unknown_gender
number of files for the gender: 16
number of entities for the gender: 18
The most popular entity: в


In [24]:
orig_ng_female_files, orig_ng_male_files, orig_ng_common_files = split_files(orig_ng_file_gender_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/ng")

### BRUK

In [23]:
orig_bruk_file_gender_dict, orig_bruk_total_job_counter, orig_bruk_job_list = return_gendered_dict(parallel_dataset_bruk_dataset, "orig_ann")
return_gender_stat(orig_bruk_total_job_counter, orig_bruk_file_gender_dict)
orig_bruk_female_files, orig_bruk_male_files, orig_bruk_common_files = split_files(orig_bruk_file_gender_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk")

3it [00:00,  6.34it/s]

unkown for: поет-символіст  -  символіст  -  символіст


86it [00:13,  6.15it/s]

unkown for: підмайстра  -  підмайстр  -  підмайстр
unkown for: підмайстра  -  підмайстр  -  підмайстр


188it [00:38,  6.05it/s]

...

unkown for: лінгвісти  -  лінгвість  -  лінгвість


480it [01:53,  3.40it/s]

unkown for: урядовці  -  урядовка  -  урядовці


492it [01:54,  4.28it/s]

total size: 638
gender: male
number of files for the gender: 122
number of entities for the gender: 508
The most popular entity: поет
gender: female
number of files for the gender: 23
number of entities for the gender: 46
The most popular entity: вчителька
gender: common
number of files for the gender: 24
number of entities for the gender: 53
The most popular entity: голова
gender: unknown_gender
number of files for the gender: 14
number of entities for the gender: 31
The most popular entity: наука


## Swapped entitites

### Ng

In [22]:
changed_ng_file_gender_dict, changed_ng_total_job_counter, changed_ng_job_list = return_gendered_dict(parallel_dataset_ng_dataset, "changed_ann", 1)
return_gender_stat(changed_ng_total_job_counter, changed_ng_file_gender_dict)
changed_ng_female_files, changed_ng_male_files, changed_ng_common_files = split_files(changed_ng_file_gender_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/ng_changed")

84it [00:16,  6.03it/s]

unkown for: академічки  -  академічка  -  академічка


150it [00:38,  6.37it/s]

unkown for: бойовички  -  бойовичка  -  бойовичка


234it [01:10,  2.04it/s]

...

unkown for: світлана зубачик  -  зубачик  -  зубачик


973it [04:42,  4.78it/s]

unkown for: в. о. управління  -  в  -  в.


1021it [04:55,  3.46it/s]

total size: 1338
gender: male
number of files for the gender: 88
number of entities for the gender: 146
The most popular entity: слідчий
gender: female
number of files for the gender: 263
number of entities for the gender: 1002
The most popular entity: директорка
gender: common
number of files for the gender: 85
number of entities for the gender: 160
The most popular entity: голова
gender: unknown_gender
number of files for the gender: 30
number of entities for the gender: 30
The most popular entity: в


### BRUK

In [21]:
changed_bruk_file_gender_dict, changed_bruk_total_job_counter, changed_bruk_job_list = return_gendered_dict(parallel_dataset_bruk_dataset, "changed_ann", 1)
return_gender_stat(changed_bruk_total_job_counter, changed_bruk_file_gender_dict)
changed_bruk_female_files, changed_bruk_male_files, changed_bruk_common_files = split_files(changed_bruk_file_gender_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk_changed")

0it [00:00, ?it/s]

2it [00:00,  3.19it/s]

unkown for: поетеса-символістка  -  символістка  -  символістка


7it [00:01,  6.62it/s]

unkown for: поетес  -  поетес  -  поетёс


...

unkown for: тренеркам  -  тренерк  -  тренерке


475it [02:05,  6.95it/s]

unkown for: академікині  -  академікина  -  академікині


492it [02:09,  3.81it/s]

total size: 638
gender: male
number of files for the gender: 44
number of entities for the gender: 80
The most popular entity: вчитель
gender: female
number of files for the gender: 112
number of entities for the gender: 445
The most popular entity: черниця
gender: common
number of files for the gender: 24
number of entities for the gender: 52
The most popular entity: голова
gender: unknown_gender
number of files for the gender: 29
number of entities for the gender: 61
The most popular entity: наука


## Collect dataset

In [33]:
def move_file(filename, destination_dir):
    file_name = os.path.basename(filename)

    destination_path = os.path.join(destination_dir, file_name)

    shutil.copy(filename, destination_path)

def move_gender_files(gender_files_list, dest_dir):
    
    for gender_file in gender_files_list:
        move_file(gender_file, dest_dir)

        ann_gender_file = gender_file.replace(".txt", ".ann")
        move_file(ann_gender_file, dest_dir)
        

### Original

Ng

In [25]:
move_gender_files(orig_ng_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_male/ng')
move_gender_files(orig_ng_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_female/ng')
move_gender_files(orig_ng_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_common/ng')

BRUK

In [26]:
move_gender_files(orig_bruk_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_male/bruk')
move_gender_files(orig_bruk_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_female/bruk')
move_gender_files(orig_bruk_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_common/bruk')

### Swapped

Ng

In [34]:
move_gender_files(changed_ng_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_male/ng_changed')
move_gender_files(changed_ng_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_female/ng_changed')
move_gender_files(changed_ng_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_common/ng_changed')

BRUK

In [29]:
move_gender_files(changed_bruk_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_male/bruk_changed')
move_gender_files(changed_bruk_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_female/bruk_changed')
move_gender_files(changed_bruk_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_common/bruk_changed')

# For PERS entities

In [160]:
female_names = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/female_fname_freq_dict.csv")
male_names = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/male_fname_freq_dict.csv")

In [162]:
female_names_list = female_names['name'].values
male_names_list = male_names['name'].values

In [ ]:
dict_df = pd.read_csv('/Users/linndfors/study/diploma/dict_uk/out/dict_corp_lt.txt', delimiter=' ', header=None, names=['word', 'lemma', 'grammar'])


In [259]:
def define_sex(name):
    try:
        filtered_df = dict_df[(dict_df['lemma'] == name) & (dict_df['word'] == name)]
        if not filtered_df.empty:
            grammar_pers = filtered_df['grammar'].iloc[0]

            if re.search(r'fname', grammar_pers):
                if re.search(r'\bf\b', grammar_pers):
                    return "F"
                elif re.search(r'\bm\b', grammar_pers):
                    return "M"
                else:
                    return "U"
            elif re.search(r'lname', grammar_pers):
                return "U"
            else:
                return "U"
        else:
            # print("No matches found for:", name)
            return "U"
    except Exception as e:
        # print("error", e)
        return "U"

In [260]:
import os
import glob

def extract_gender(entity):

    pers_parts = entity.split(" ")
    for i in pers_parts:
        if define_sex(i) == "F":
            return "female", i
        elif define_sex(i) == "M":
            return "male", i
        
    doc = nlp(entity)
    
    words = {word.lemma for sentence in doc.sentences for word in sentence.words}
    words.update(entity.split(" "))

    for x in words:
        if x in female_names_list:
            return "female", x
        if x in male_names_list:
            return "male", x
        
    # print("unkown for:", entity)
    return "unknown_gender", entity

def extract_pers_entities(directory):
    pers_entities = {}
    
    ann_files = glob.glob(os.path.join(directory, "*.ann"))
    print(ann_files)
    
    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) == 5:
                    
                    entity_type = parts[1]
                    
                    if entity_type == "PERS":
                        entity_text = parts[4]
                        if ann_file in pers_entities:
                            pers_entities[ann_file].append(entity_text)
                        else:
                            pers_entities[ann_file] = [entity_text]
    
    return pers_entities

In [235]:
def return_gendered_PERS_dict(pers_dict):
    file_gender_dict = {"male": {}, "female": {}, "unknown_gender": {}}

    total_pers_counter = 0
    pers_list = []

    for filename, values in tqdm.tqdm(pers_dict.items()):
        filename = filename.replace(".ann", ".txt")
        
        try:
            for ent in values:
                total_pers_counter += 1
                pers_list.append(ent)
                if len(ent.split(' ')) > 1:
                    ent = ent.split(' ')[0]
                gender_value, lemma_word = extract_gender(ent)
                
                if filename not in file_gender_dict[gender_value]:
                    file_gender_dict[gender_value][filename] = [(ent, lemma_word)]
                else:
                    file_gender_dict[gender_value][filename].append((ent, lemma_word))
                    
        except Exception as e:
            print(f"Issue with row: {row} - Error: {e}")
    return file_gender_dict, total_pers_counter, pers_list

In [ ]:
changed_bruk_pers_dict = extract_pers_entities("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed")
changed_bruk_file_gender_pers_dict, changed_bruk_total_pers_counter, changed_bruk_pers_list = return_gendered_PERS_dict(changed_bruk_pers_dict)

In [288]:
return_gender_stat(changed_bruk_total_pers_counter, changed_bruk_file_gender_pers_dict)
changed_bruk_pers_female_files, changed_bruk_pers_male_files, changed_bruk_pers_common_files = split_files(changed_bruk_file_gender_pers_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed")

total size: 326
gender: male
number of files for the gender: 32
number of entities for the gender: 52
The most popular entity: Сергій
gender: female
number of files for the gender: 67
number of entities for the gender: 132
The most popular entity: Олександра
gender: unknown_gender
number of files for the gender: 48
number of entities for the gender: 142
The most popular entity: А


In [ ]:
# changed_bruk_female_files, changed_bruk_male_files, changed_bruk_common_files = split_files(changed_bruk_file_gender_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed")

In [244]:
changed_ng_pers_dict = extract_pers_entities("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed")
changed_ng_file_gender_pers_dict, changed_ng_total_pers_counter, changed_ng_pers_list = return_gendered_PERS_dict(changed_ng_pers_dict)

100%|██████████| 253/253 [39:13<00:00,  9.30s/it]   


In [286]:
return_gender_stat(changed_ng_total_pers_counter, changed_ng_file_gender_pers_dict)
changed_ng_pers_female_files, changed_ng_pers_male_files, changed_ng_pers_common_files = split_files(changed_ng_file_gender_pers_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed")

total size: 1058
gender: male
number of files for the gender: 106
number of entities for the gender: 170
The most popular entity: Валентин
gender: female
number of files for the gender: 225
number of entities for the gender: 602
The most popular entity: Олена
gender: unknown_gender
number of files for the gender: 145
number of entities for the gender: 286
The most popular entity: Світлани


In [276]:
all_names = []

for filaname, ents in changed_ng_file_gender_pers_dict['unknown_gender'].items():
    for ent in ents:
        all_names.append(ent[0])

In [ ]:
name_counts = Counter(all_names)
print(name_counts)

Counter({'Світлани': 7, 'Юлії': 7, 'Людмили': 5, 'Любові': 5, 'Павленко': 4, 'Олені': 4, 'Козименко': 4, 'Кузнєцова': 3, 'Непийвода': 3, 'Машненкова': 3, 'Лілії': 3, 'Ірині': 3, 'Кобринчук': 3, 'Янукович': 3, 'Андреїшин': 3, 'Кушнір': 3, 'Сміяненко': 3, 'Тарпан': 2, 'Філіпчук': 2, 'Ніколєнко': 2, 'Тютюнник': 2, 'Ганну': 2, 'Ріната': 2, 'Ярич': 2, 'Пелих': 2, 'Ніні': 2, 'Осипенко': 2, 'Кокседж': 2, 'Машненкову': 2, 'Наталію': 2, 'Борзих': 2, 'Корнієць': 2, 'Чепурненко': 2, 'Владислави': 2, 'Фірташа': 2, 'Ігоря': 2, 'Василенко': 2, 'Смик': 2, 'Любач': 2, 'Біловол': 2, 'Юлію': 2, 'Ковач': 2, 'Підвисоцька': 2, 'Шишацька': 2, 'Голяшкіної': 2, 'Григорія': 2, 'Тертиця': 2, 'Колуги': 2, 'Чернікова': 2, 'Васильєвою': 1, 'Кисельовою': 1, 'Котельникову': 1, 'Довгополова': 1, 'Ватерина': 1, 'Демішкан': 1, 'Сагайдак': 1, 'Остапенко': 1, 'Анісімових': 1, 'Котляров': 1, 'Анісімова': 1, 'Ахметова': 1, 'Анжеліці': 1, 'Кусок': 1, 'Медведчук': 1, 'Овчаренко': 1, 'Аблова': 1, 'Червачова': 1, 'Качаненку': 

In [261]:
bruk_pers_dict = extract_pers_entities("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk")
bruk_file_gender_pers_dict, bruk_total_pers_counter, bruk_pers_list = return_gendered_PERS_dict(bruk_pers_dict)

['/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/68f6d1cfc486.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/dc25535e13d7.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/fad2422a7bf3.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/7e1defe0dda2.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/87ae82667e2a.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/b972a141f2be.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/85622bc925dc.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/36ac3bc5ed9c.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/3d0ab8ae38cf.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk/ead06d34c49d.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/

100%|██████████| 231/231 [2:59:33<00:00, 46.64s/it]  


In [284]:
return_gender_stat(bruk_total_pers_counter, bruk_file_gender_pers_dict)
orig_bruk_pers_female_files, orig_bruk_pers_male_files, orig_bruk_pers_common_files = split_files(bruk_file_gender_pers_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk")

total size: 4415
gender: male
number of files for the gender: 176
number of entities for the gender: 1274
The most popular entity: Андрій
gender: female
number of files for the gender: 135
number of entities for the gender: 970
The most popular entity: Аліна
gender: unknown_gender
number of files for the gender: 200
number of entities for the gender: 2171
The most popular entity: Бог


In [263]:
ng_pers_dict = extract_pers_entities("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng")
ng_file_gender_pers_dict, ng_total_pers_counter, ng_pers_list = return_gendered_PERS_dict(ng_pers_dict)

['/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/b4fe41ad2268.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/150446f83aa2.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/ee7cde9751a7.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/3fdf22393022.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/76b9b07172fb.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/7982679365fb.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/003d28360166.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/02634352df22.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/a9d2c035032d.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/3eef4bd7fc75.ann', '/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NE

100%|██████████| 279/279 [1:09:37<00:00, 14.97s/it]


In [278]:
return_gender_stat(ng_total_pers_counter, ng_file_gender_pers_dict)
orig_ng_pers_female_files, orig_ng_pers_male_files, orig_ng_pers_common_files = split_files(ng_file_gender_pers_dict, "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng")

total size: 1820
gender: male
number of files for the gender: 252
number of entities for the gender: 846
The most popular entity: Сергій
gender: female
number of files for the gender: 158
number of entities for the gender: 316
The most popular entity: Олена
gender: unknown_gender
number of files for the gender: 205
number of entities for the gender: 658
The most popular entity: Ігоря


In [281]:
len(orig_ng_pers_male_files)

252

### Move files

orig ng

In [ ]:
move_gender_files(orig_ng_pers_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_male/ng')
move_gender_files(orig_ng_pers_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_female/ng')
# move_gender_files(orig_ng_pers_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_common/ng')

orig bruk

In [ ]:
move_gender_files(orig_bruk_pers_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_male/bruk')
move_gender_files(orig_bruk_pers_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_female/bruk')
# move_gender_files(orig_bruk_pers_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_common/bruk')

swapped ng

In [287]:
move_gender_files(changed_ng_pers_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_male/ng_changed')
move_gender_files(changed_ng_pers_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_female/ng_changed')
# move_gender_files(changed_ng_pers_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_common/ng')

swapped bruk

In [290]:
move_gender_files(changed_bruk_pers_male_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_male/bruk_changed')
move_gender_files(changed_bruk_pers_female_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_female/bruk_changed')
# move_gender_files(orig_bruk_pers_common_files, '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_pers/data_for_ner_common/bruk')